In [5]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import os 
import time

def wikiscraper(team_name, year_int):
    '''This wikiscraper takes the team name string and year integer and returns a list of that team's men's basketball roster for that year'''
    # team ex: "Illinois Fighting Illini"
    # year_int ex: 2024
    team = team_name.replace(" ", "_")
    year = str(year_int-1) + "-" + str(year_int%2000)
    # csv_folder = "csv_files"
    # year_path = os.path.join(csv_folder, year)
    # csv_file_path = os.path.join(year_path, f'{team}_{year}.csv')

    # if os.path.exists(csv_file_path):
    #     return
    try: 
        roster = f"https://en.wikipedia.org/wiki/{year}_{team}_men%27s_basketball_team"
        result = requests.get(roster)
        if(result.status_code == 404):
            roster = f"https://en.wikipedia.org/wiki/{year}_{team}_basketball_team"
            result = requests.get(roster)
            if(result.status_code == 404):
                print('error ' + team_name + " " + year)
                return
        content = result.text

        soup = BeautifulSoup(content, 'lxml')
        table = soup.find('table', class_= 'toccolours')
        table = table.find('table', class_='sortable')

        player_data = []

        rows = table.find_all('tr')
        for row in rows[1:]:
            cells = row.find_all('td')
            name = cells[2].text.strip().replace('\xa0(W)', '').split('(')[0].strip().split('[')[0].strip()
            try:
                hometown = cells[7].text.strip()
            except:
                hometown = cells[6].text.strip()
            player_data.append({'Team': team_name, 'Year': year_int, 'Name': name, 'Hometown': hometown})
    except:
        print("error thrown for " + team_name + " " + year)
        return

    # if not os.path.exists(csv_folder):
    #     os.makedirs(csv_folder)

    # if not os.path.exists(year_path):
    #     os.makedirs(year_path)

    # df.to_csv(csv_file_path, index=False)
    print(f'{team_name} CSV uploaded!')
    time.sleep(2)
    return player_data

In [6]:
bigten = {"Penn State Nittany Lions", "Iowa Hawkeyes", "Illinois Fighting Illini", "Ohio State Buckeyes", "Wisconsin Badgers", "Purdue Boilermakers", "Indiana Hoosiers", "Michigan Wolverines", "Michigan State Spartans", "Nebraska Cornhuskers", "Northwestern Wildcats", "Minnesota Golden Gophers",  "Maryland Terrapins", "Rutgers Scarlet Knights"}
acc = {"Clemson Tigers", "Duke Blue Devils", "North Carolina Tar Heels", "Wake Forest Demon Deacons", "NC State Wolfpack", "Boston College Eagles", "Syracuse Orange", "Florida State Seminoles", "Louisville Cardinals", "Virginia Cavaliers", "Virginia Tech Hokies", "Miami Hurricanes", "Georgia Tech Yellow Jackets", "Pittsburgh Panthers"}
bigten_data = []
acc_data = [] 

teams = {"TBD"}
team_data = []
for team in teams:
    for year in range (2024,2014,-1):
        # wikiscraper(team,year)
        team_data.extend(wikiscraper(team, year))

df = pd.DataFrame(team_data)
df.to_csv('Teams_Data.csv', index=False)

# will take like 11 minutes to run. to prevent getting blocked from the server

Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!
Illinois Fighting Illini CSV uploaded!


In [7]:
import pandas as pd
def stateconvert(key_code):
    state_codes_to_full = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'DC': 'District of Columbia',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
        'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
        'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
        'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
        'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
        'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
        'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'ON': 'Ontario', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'P.R.': 'Puerto Rico', 'PR': 'Puerto Rico'
    }
    return state_codes_to_full.get(key_code, key_code)

df = pd.read_csv('Teams_Data.csv')
df['City'] = df['Hometown'].str.split(',').str[0].str.strip()
df['State'] = df['Hometown'].str.split(',').str[1].str.strip().apply(stateconvert)
df['Hometown'] = df['City'] + ', ' + df['State']

df.to_csv('Teams_Data.csv', index=False)


In [8]:
from geopy.geocoders import ArcGIS
import pandas as pd

# Load CSV data
df_b10 = pd.read_csv('csvfiles/Pac_12_Data.csv')

# Initialize ArcGIS geocoder
nom = ArcGIS()

# Dictionary to cache geocoding results
cache = {}
index = 0
# Function to get latitude
def get_latitude(city):
    city = str(city)
    global cache
    if city in cache:
        return cache[city]['latitude']
    else:
        location = nom.geocode(city)
        if location:
            cache[city] = {'latitude': location.latitude, 'longitude': location.longitude}
            return location.latitude
        else:
            return None

# Function to get longitude
def get_longitude(city):
    city = str(city)
    global cache
    global index
    index +=1
    print(index)
    if city in cache:
        return cache[city]['longitude']
    else:
        location = nom.geocode(city)
        if location:
            cache[city] = {'latitude': location.latitude, 'longitude': location.longitude}
            return location.longitude
        else:
            return None

# Apply functions to DataFrame
df_b10['Latitude'] = df_b10['Hometown'].apply(get_latitude)
df_b10['Longitude'] = df_b10['Hometown'].apply(get_longitude)

df_b10.to_csv('Teams_Data.csv', index=False)

KeyboardInterrupt: 

In [17]:
df_b10.to_csv('ACC_Data_Final.csv', index=False)